<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# OpenEO Platform - Client Side Processing


In this interactive notebook we will show some usage eaxample of the Client Side Processing functionality added recently to the openEO Python client.

## Requirements

To use this functionality, you need `3.9<=Python<3.11`.

You can install it using:
    `pip install openeo[localprocessing]`

<div class="alert alert-block alert-warning">
This functionality is still under development and the installation procedure might change.
Please refer to official documentation page for the most up to date instructions:
    
<a href="url">https://open-eo.github.io/openeo-python-client/cookbook/localprocessing.html</a>

## Sample Datasets

Clone the repository containing sample datasets provided by openEO Platform:

In [ ]:
import os
if not os.path.exists('./openeo-localprocessing-data'):
    !git clone https://github.com/Open-EO/openeo-localprocessing-data.git

Initialize the local collections and inspect them:

In [ ]:
from openeo.local import LocalConnection
local_conn = LocalConnection(['./openeo-localprocessing-data'])
local_conn.list_collections()

## Example 1: Create color composites based on Sentinel-2 data

The first step, as usual in an openEO pipeline, is `load_collection`. In this case, the collection id is the same as the file path we want to use:

In [ ]:
cube = local_conn.load_collection('openeo-localprocessing-data/sample_netcdf/S2_L2A_sample.nc').filter_bands(['B04','B03','B02'])

We would like to create a color composite based on multiple acquisitions over time.

For doing so, we will use the `reduce_dimension` process, which can reduce a dimension given a reducer process.

In this case the dimension is the temporal (`t`) and the reducer is `mean`, to get a temporal average.

In [ ]:
cube_avg = cube.reduce_dimension(dimension='t',reducer='mean')

Since we would like to visualize the result as an image, we scale it to values between 0 and 255.

In [ ]:
cube_scaled = cube_avg.linear_scale_range(0,2000,0,255)

We can now visualize the openEO process graph that we just created:

In [ ]:
cube_scaled

By calling `.execute()`, the process graph will be locally executed and the result will be returned as an Xarray object:

In [ ]:
result = cube_scaled.execute()
result

Now that we have the result, we can plot the color composites:

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1,figsize=(7.5,5))
result.plot.imshow(ax=ax,cmap='brg',vmin=0,vmax=255)
ax.set_title('Mean RGB composite - June 2022 - Bolzano, Italy - Sentinel-2')
plt.show()

We immediately notice that something is wrong. Probably some days where cloudy! Therefore, the average image is cloudy as well.

It is possible can solve this issue with two different approaches:
1. Trying to use a different reducer. We will try to use `median` instead of `mean`.
2. By applying a cloud mask, based on the Sentinel-2 SCL (Scene Classification Layer) band. (Not possible yet locally, some processes missing)

Let's try the first approach, using `median` instead of `mean`.

In [ ]:
cube = local_conn.load_collection('openeo-localprocessing-data/sample_netcdf/S2_L2A_sample.nc').filter_bands(['B04','B03','B02'])
cube_med = cube.reduce_dimension(dimension='t',reducer='median')
cube_scaled = cube_med.linear_scale_range(0,2000,0,255)
result_median = cube_scaled.execute()

We can now compare the results:

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,2,figsize=(15,5))
result.plot.imshow(ax=ax[0],cmap='brg',vmin=0,vmax=255)
ax[0].set_title('Mean RGB composite - June 2022 - Bolzano, Italy - Sentinel-2')
result_median.plot.imshow(ax=ax[1],cmap='brg',vmin=0,vmax=255)
ax[1].set_title('Median RGB composite - June 2022 - Bolzano, Italy - Sentinel-2')
plt.show()

We could now try to more advanced method, based on the SCL layer:

In [ ]:
cube = local_conn.load_collection('openeo-localprocessing-data/sample_netcdf/S2_L2A_sample.nc')

We now want to mask out the SCL class 9 (CLOUD_HIGH_PROBABILITY):

In [ ]:
scl = cube.band('SCL')
cloud_mask_high =  scl != 9
cloud_mask_medium =  scl != 8
cloud_mask = (cloud_mask_high + cloud_mask_medium) == 2

We can now apply the cloud mask to the input data:

In [ ]:
cube_masked = cube.filter_bands(['B04','B03','B02']).merge_cubes(cloud_mask,overlap_resolver='multiply')

Check if the cloud mask is applied correctly:

In [ ]:
cube_masked.execute()[:,0].plot.imshow(vmin=0,vmax=2500)

And finally take the mean over time, rescale and execute:

In [ ]:
cube_masked = cube_masked.reduce_dimension(dimension='t',reducer='median').linear_scale_range(0,2000,0,255)
cube_masked

In [ ]:
result_masked = cube_masked.execute()

Finally visualize the result and compare with the median version with no cloud filter:

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,2,figsize=(15,5))
result_median.plot.imshow(ax=ax[0],cmap='brg',vmin=0,vmax=255)
ax[0].set_title('Median RGB composite - June 2022 - Bolzano, Italy - Sentinel-2')
result_masked.plot.imshow(ax=ax[1],cmap='brg',vmin=0,vmax=255)
ax[1].set_title('Median RGB composite cloud masked - June 2022 - Bolzano, Italy - Sentinel-2')
plt.show()